# **Week-4 Live Session-1**

In [1]:
import numpy as np
import pandas as pd

### 1. Get The Data

In [3]:
data = pd.read_csv('train.csv')

In [4]:
data.shape

(9800, 18)

In [ ]:
data.head(2)

In [ ]:
data.info()

In [ ]:
data.isin(['?', -1]).sum()

In [ ]:
data.isnull().sum()

In [ ]:
data.nunique()

In [5]:
data['Order Date'] = pd.to_datetime(data['Order Date'], dayfirst = True)
data['Ship Date'] = pd.to_datetime(data['Ship Date'], dayfirst = True)

In [ ]:
data.info()

In [7]:
data['order year'] = data['Order Date'].dt.year

In [8]:
data['Ship Year'] = data['Ship Date'].dt.year

In [9]:
data.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'order year', 'Ship Year'],
      dtype='object')

In [ ]:
# Order in the month of july

data[data['Order Date'].dt.month == 7][data['Order Date'].dt.year == 2016]

In [10]:
# Unimportant features

unimp = ['Row ID', 'Order ID', 'Order Date', 'Ship Date','Customer ID', 'Customer Name','Country','Product ID','Product Name']

In [11]:
data.drop(unimp, axis = 1, inplace=True)

In [12]:
data.columns

Index(['Ship Mode', 'Segment', 'City', 'State', 'Postal Code', 'Region',
       'Category', 'Sub-Category', 'Sales', 'order year', 'Ship Year'],
      dtype='object')

In [13]:
data.head()

,Ship Mode,Segment,City,State,Postal Code,Region,Category,Sub-Category,Sales,order year,Ship Year
0,Second Class,Consumer,Henderson,Kentucky,42420.0,South,Furniture,Bookcases,261.9600,2017,2017
1,Second Class,Consumer,Henderson,Kentucky,42420.0,South,Furniture,Chairs,731.9400,2017,2017
2,Second Class,Corporate,Los Angeles,California,90036.0,West,Office Supplies,Labels,14.6200,2017,2017
3,Standard Class,Consumer,Fort Lauderdale,Florida,33311.0,South,Furniture,Tables,957.5775,2016,2016
4,Standard Class,Consumer,Fort Lauderdale,Florida,33311.0,South,Office Supplies,Storage,22.3680,2016,2016


In [ ]:
data.isnull().sum()

In [14]:
from sklearn.model_selection import train_test_split

In [41]:
X = data.drop(['Sales'], axis = 1)
y = data['Sales']

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [43]:
X_train['Postal Code'].value_counts().index[:8]

Float64Index([10035.0, 10024.0, 10009.0, 10011.0, 94122.0, 19134.0, 98105.0,
              94110.0],
             dtype='float64')

In [ ]:
# top 8 places for my dataset

#[10035.0, 10024.0, 10009.0, 94122.0, 10011.0, 94110.0, 98105.0, 19134.0, 'other']

In [ ]:
X_train['Postal Code'][~X_train['Postal Code'].isin([10035.0, 10024.0, 10009.0, 94122.0, 10011.0, 94110.0, 98105.0, 19134.0])] = 0

In [68]:
X_train['Postal Code'].value_counts()

0.0        5757
10035.0     183
10024.0     156
10009.0     153
10011.0     131
94122.0     129
19134.0     121
98105.0     119
94110.0     111
Name: Postal Code, dtype: int64

In [ ]:
X_train.select_dtypes(exclude = 'number').nunique()

In [ ]:
X_train['City'].value_counts()

In [ ]:
X_train['State'].value_counts()

In [49]:
X_train['State'].value_counts().index[:8]

Index(['California', 'New York', 'Texas', 'Pennsylvania', 'Washington',
       'Illinois', 'Ohio', 'Florida'],
      dtype='object')

In [50]:
def replace_values (df, column_name):
  df[column_name][~df[column_name].isin(df[column_name].value_counts().index[:8])] = 'Other'
  return df

In [ ]:
replace_values(X_train, 'City')
replace_values(X_train, 'State')
replace_values(X_train, 'Sub-Category')

In [53]:
X_train.select_dtypes(exclude = 'number').nunique()

Ship Mode       4
Segment         3
City            9
State           9
Postal Code     9
Region          4
Category        3
Sub-Category    9
dtype: int64

In [57]:
X_train.select_dtypes('number').columns

Index(['order year', 'Ship Year'], dtype='object')

In [55]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer

In [69]:
cat_cols = ['Ship Mode', 'Segment', 'City', 'State', 'Region', 'Postal Code', 'Category', 'Sub-Category']
num_cols = ['order year', 'Ship Year']

In [70]:
transf = [
    ('ohe', OneHotEncoder(sparse_output = False), cat_cols),
    ('scale', MinMaxScaler(), num_cols)
]

ct = ColumnTransformer(transformers = transf, remainder = 'passthrough')


In [71]:
X_transformed = ct.fit_transform(X_train)

In [ ]:
pd.DataFrame(ct.fit_transform(X_train))

In [73]:
X_transformed.shape

(6860, 52)

In [74]:
y_train.shape

(6860,)

# **Week-4 Live Session-2**

### Feature Selection

In [81]:
from sklearn.feature_selection import VarianceThreshold, SelectKBest, mutual_info_regression, f_regression

In [80]:
selector = VarianceThreshold(threshold = 0.1)
pd.DataFrame(selector.fit_transform(X_transformed))

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.666667
1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000000
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.000000
3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.000000
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6855,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.000000
6856,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.000000
6857,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.000000
6858,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000000


In [82]:
f_regression(X_transformed, y_train)

(array([5.49283129e-03, 3.07860418e-01, 3.10706545e-01, 1.98345537e-02,
        2.21133332e-01, 1.16911350e-01, 4.09922210e-02, 6.01836278e+00,
        9.52237717e-01, 2.15253783e+00, 4.50172150e-01, 6.95647901e+00,
        2.64095803e-02, 4.50152673e+00, 9.67555223e-03, 6.03144980e+00,
        2.89993796e-02, 2.34022559e+00, 5.85087948e+00, 6.73154697e+00,
        1.10790107e+00, 3.46485942e+00, 5.29117424e+00, 6.61472022e+00,
        6.53788908e+00, 1.67896199e-03, 2.50210637e-02, 7.07790929e-01,
        6.09479760e-01, 4.80104317e+00, 3.97730287e-01, 8.80832817e-01,
        2.72396921e+00, 3.23922225e+00, 9.04467918e-01, 5.21066042e-02,
        4.99632003e-01, 6.47648526e-01, 9.02855362e+01, 3.57756003e+02,
        1.82198157e+02, 7.15881765e-02, 6.52340183e+01, 2.74229759e+01,
        1.26640031e+02, 3.58431458e+01, 8.97814355e+01, 9.34663025e+01,
        4.64054792e+01, 4.57781045e+00, 8.31846874e-01, 1.04740725e+00]),
 array([9.40922125e-01, 5.79014191e-01, 5.77265136e-01, 8.8800

In [83]:
skb = SelectKBest(f_regression, k=10)
skb.fit_transform(X_transformed, y_train)

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 1., 0., 0.]])

In [84]:
pd.DataFrame(skb.fit_transform(X_transformed, y_train))

,0,1,2,3,4,5,6,7,8,9
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
6855,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6856,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6857,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6858,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [86]:
skb.get_feature_names_out()

array(['x38', 'x39', 'x40', 'x42', 'x43', 'x44', 'x45', 'x46', 'x47',
       'x48'], dtype=object)